In [1]:
from keras.layers import Convolution2D

Using TensorFlow backend.


In [2]:
from keras.layers import MaxPooling2D

In [3]:
from keras.layers import Flatten

In [4]:
from keras.layers import Dense

In [5]:
from keras.models import Sequential

In [6]:
model = Sequential()

In [7]:
model.add(Convolution2D(filters=32, 
                        kernel_size=(3,3), 
                        activation='relu',
                   input_shape=(64, 64, 3)
                       ))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [10]:
model.add(Convolution2D(filters=32, 
                        kernel_size=(3,3), 
                        activation='relu',
                       ))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.add(Flatten())

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.add(Dense(units=128, activation='relu'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
Total params: 813,088
Trainable params: 813,088
Non-trainable params: 0
________________________________________________

In [17]:
model.add(Dense(units=5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from keras_preprocessing.image import ImageDataGenerator

In [21]:
traingen = ImageDataGenerator(rescale=1./255 , zoom_range=0.2 , horizontal_flip=True ,  vertical_flip=True )

In [22]:
testgen = ImageDataGenerator(rescale=1./255)

In [25]:
trainset = traingen.flow_from_directory("C:/Users/Mukul J'veriya/Desktop/mlops-ws/cnn_dataset/training_set" , target_size=(64,64) , batch_size=32 , class_mode='categorical' )

Found 9259 images belonging to 5 classes.


In [26]:
testset = testgen.flow_from_directory("C:/Users/Mukul J'veriya/Desktop/mlops-ws/cnn_dataset/test_set/" , target_size=(64,64) , batch_size=32 , class_mode='categorical' )

Found 3107 images belonging to 5 classes.


In [28]:
model.fit(trainset , epochs=2 , validation_data=testset , steps_per_epoch=9259, validation_steps=10  )
model.save("cnnmodel.h5")

Epoch 1/2
9259/9259 [==============================] - 2318s 250ms/step - loss: 0.4795 - accuracy: 0.7743 - val_loss: 0.3930 - val_accuracy: 0.7563
Epoch 2/2
9259/9259 [==============================] - 2271s 245ms/step - loss: 0.2747 - accuracy: 0.8826 - val_loss: 1.5967 - val_accuracy: 0.7750


In [44]:
from keras.models import load_model

classifier = load_model('cnnmodel.h5')

In [49]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

cnn_dataset_dict = {"[0]": "aeroplan", 
                      "[1]": "bike",
                      "[2]": "cats",
                      "[3]": "dogs",
                      "[4]": "flower",
                      }

cnn_dataset_dict_n = {"aeroplan": "Aeroplan", 
                      "bike": "Bike",
                      "cats": "Cats",
                      "dogs": "Dogs",
                      "flower": "Flower",
                      }


def draw_test(name, pred, im):
    monkey = cnn_dataset_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + cnn_dataset_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C:/Users/Mukul J'veriya/Desktop/mlops-ws/cnn_dataset/test_set/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (64, 64), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,64,64,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Dogs
Class - Dogs
Class - Bike
Class - Bike
Class - Flower
Class - Cats
Class - Flower
Class - Bike
Class - Flower
Class - Flower


In [50]:
trainset.class_indices

{'aeroplan': 0, 'bike': 1, 'cats': 2, 'dogs': 3, 'flower': 4}